# Установка зависимостей

In [2]:
# 0. Обновление pip и инструментов сборки
%pip install -U pip setuptools wheel

# 4. Установка утилит
%pip install -U scikit-learn pandas networkx matplotlib rich tqdm
%pip install graphlime tensorflow

%pip install torch==2.6.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
%pip install torch_geometric
%pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.6.0+cu126.html




Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu126
  Using cached https://download.pytorch.org/whl/cu126/torch-2.6.0%2Bcu126-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached https://download.pytorch.org/whl/cu126/torchvision-0.22.0%2Bcu126-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/cu126/torchaudio-2.7.0%2Bcu126-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Using cached https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Using cached https://download.pytorch.org/whl/mpmath-1.3.0-py3-none-any.whl (536 kB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with ot

# Загрузка датасета

In [5]:
#@title 📂 Загрузка датасета (идемпотентно)
import os

# Папка, куда будет сохранён репозиторий
data_dir = 'data/'

import os
import shutil

data_dir = 'data/'

# Проверяем, существует ли папка
if os.path.exists(data_dir):
    # Если папка существует, удаляем все её содержимое
    for root, dirs, files in os.walk(data_dir, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))
else:
    # Если папка не существует, создаем её
    os.makedirs(data_dir)


# Клонирование репозитория в папку data
!git clone https://github.com/salam-ammari/Labeled-Transactions-based-Dataset-of-Ethereum-Network.git {data_dir}


import zipfile, pathlib
zip_path = "data/Dataset.zip"
extract_path = pathlib.Path("data/unpacked")
with zipfile.ZipFile(zip_path) as zf: zf.extractall(extract_path)
print("✅ Распаковка завершена")

✅ Распаковка завершена


Cloning into 'data'...


In [16]:
#@title 🧹 Чтение CSV
import pandas as pd
dataset_csv_path = extract_path / "Dataset" / "Dataset.csv"
df = pd.read_csv(dataset_csv_path)
print(df.shape, "строк")
print(df.columns.tolist())

(71250, 18) строк
['hash', 'nonce', 'transaction_index', 'from_address', 'to_address', 'value', 'gas', 'gas_price', 'input', 'receipt_cumulative_gas_used', 'receipt_gas_used', 'block_timestamp', 'block_number', 'block_hash', 'from_scam', 'to_scam', 'from_category', 'to_category']


# Инженерия данных

In [17]:
# %% [data scrubbing]
# Список столбцов, в которых НЕ хотим допускать NaN
cols_required = [col for col in df.columns 
                 if col not in ['from_category', 'to_category']]

# Считаем пропуски по этим столбцам
missing_counts_req = df[cols_required].isnull().sum()
print("Missing values in required columns:\n", missing_counts_req)

# Дропаем строки, где есть хоть одно NaN в обязательных столбцах
df = df.dropna(subset=cols_required)
print("After dropping rows with missing in required cols:", df.shape)

# Проверка и дроп дубликатов
dup_count = df.duplicated().sum()
print("Duplicate rows count:", dup_count)
if dup_count > 0:
    df = df.drop_duplicates()
    print("Dropped duplicate rows. New shape:", df.shape)
else:
    print("No duplicate rows found.")

Missing values in required columns:
 hash                           0
nonce                          0
transaction_index              0
from_address                   0
to_address                     0
value                          0
gas                            0
gas_price                      0
input                          0
receipt_cumulative_gas_used    0
receipt_gas_used               0
block_timestamp                0
block_number                   0
block_hash                     0
from_scam                      0
to_scam                        0
dtype: int64
After dropping rows with missing in required cols: (71250, 18)
Duplicate rows count: 11
Dropped duplicate rows. New shape: (71239, 18)


In [18]:
# Считаем количество 0 и 1 в from_scam / to_scam
from_counts = df['from_scam'].value_counts().rename({0: 'non_scam', 1: 'scam'})
to_counts   = df['to_scam'  ].value_counts().rename({0: 'non_scam', 1: 'scam'})

print("Sender (from_scam):")
print(from_counts)
print("\nRecipient (to_scam):")
print(to_counts)

# Если вас интересуют транзакции, где есть хоть одна пометка scam:
df['any_scam'] = ((df['from_scam'] == 1) | (df['to_scam'] == 1)).astype(int)
any_counts = df['any_scam'].value_counts().rename({0: 'no_scam', 1: 'has_scam'})
print("\nTransactions with any scam flag:")
print(any_counts)

Sender (from_scam):
from_scam
non_scam    68622
scam         2617
Name: count, dtype: int64

Recipient (to_scam):
to_scam
non_scam    59601
scam        11638
Name: count, dtype: int64

Transactions with any scam flag:
any_scam
no_scam     57000
has_scam    14239
Name: count, dtype: int64


In [ ]:
import pandas as pd
import numpy as np

# # Допустим, df уже загружен и очищен от NaN/дубликатов

# # 1) Балансировка по from_scam
# n_from_scam = df['from_scam'].sum()  # должно быть 2617
# df_from_pos = df[df['from_scam'] == 1]
# df_from_neg = df[df['from_scam'] == 0].sample(n=n_from_scam, random_state=42)
# df_balanced_from = pd.concat([df_from_pos, df_from_neg]) \
#                      .sample(frac=1, random_state=42) \
#                      .reset_index(drop=True)

# print("Balanced from_scam:")
# print(df_balanced_from['from_scam'].value_counts())

# # 2) Балансировка по to_scam
# n_to_scam = df['to_scam'].sum()  # 11638
# df_to_pos = df[df['to_scam'] == 1]
# df_to_neg = df[df['to_scam'] == 0].sample(n=n_to_scam, random_state=42)
# df_balanced_to = pd.concat([df_to_pos, df_to_neg]) \
#                    .sample(frac=1, random_state=42) \
#                    .reset_index(drop=True)

# print("\nBalanced to_scam:")
# print(df_balanced_to['to_scam'].value_counts())

# Если нужно ровно 2617 для обоих (независимо от фактического числа):
# desired_n = 2617

# # По from_scam
# df_from_pos = df[df['from_scam'] == 1]
# df_from_neg = df[df['from_scam'] == 0].sample(n=desired_n, random_state=42)
# df_sampled_from_exact = pd.concat([df_from_pos, df_from_neg]) \
#                            .sample(frac=1, random_state=42) \
#                            .reset_index(drop=True)

# # По to_scam
# df_to_pos = df[df['to_scam'] == 1].sample(n=desired_n, random_state=42)
# df_to_neg = df[df['to_scam'] == 0].sample(n=desired_n, random_state=42)
# df_sampled_to_exact = pd.concat([df_to_pos, df_to_neg]) \
#                          .sample(frac=1, random_state=42) \
#                          .reset_index(drop=True)

# print("\nExact 2617 each for from_scam:")
# print(df_sampled_from_exact['from_scam'].value_counts())
# print("\nExact 2617 each for to_scam:")
# print(df_sampled_to_exact['to_scam'].value_counts())


# # Предполагаем, что df уже загружен и очищен от NaN/дубликатов
# desired_n = 2617

# # 1) Баланс по from_scam
# pos_from = df[df["from_scam"] == 1]
# neg_from = df[df["from_scam"] == 0].sample(n=desired_n, random_state=42)

# df_from_balanced = (
#     pd.concat([pos_from, neg_from])
#       .sample(frac=1, random_state=42)
#       .reset_index(drop=True)
# )

# print("FROM balance:\n", df_from_balanced["from_scam"].value_counts())

# # 2) Баланс по to_scam без пересечений с df_from_balanced
# pos_to = df[df["to_scam"] == 1].sample(n=desired_n, random_state=42)

# # из кандидатов на negative-to исключаем уже взятые в df_from_balanced
# neg_to_candidates = df[df["to_scam"] == 0]
# neg_to_candidates = neg_to_candidates.loc[
#     ~neg_to_candidates.index.isin(df_from_balanced.index)
# ]

# neg_to = neg_to_candidates.sample(n=desired_n, random_state=42)

# df_to_balanced = (
#     pd.concat([pos_to, neg_to])
#       .sample(frac=1, random_state=42)
#       .reset_index(drop=True)
# )

# print("\nTO balance:\n", df_to_balanced["to_scam"].value_counts())

# # 3) Проверка пересечений индексов между двумя наборами
# intersection = set(df_from_balanced.index) & set(df_to_balanced.index)
# print("\nПересечение индексов:", intersection)

# # 4) Если нужен единый датасет — объединяем и удаляем дубликаты
# df_balanced = (
#     pd.concat([df_from_balanced, df_to_balanced])
#       .drop_duplicates()
#       .reset_index(drop=True)
# )
# print("\nИтоговый сбалансированный датасет:", df_balanced.shape)

SEED = 42
rng = np.random.default_rng(SEED)

pos_nodes = node_label[node_label == 1].index
neg_nodes = node_label[node_label == 0].index

n_pos = len(pos_nodes)
neg_sample = rng.choice(neg_nodes, size=n_pos, replace=False)

use_nodes = pd.Index(pos_nodes).union(neg_sample)

# оставляем только выбранные адреса
node_df_bal = node_df.loc[use_nodes]
node_label_bal = node_label.loc[use_nodes]

# чистим рёбра — обе стороны должны быть внутри use_nodes
edge_df_bal = edge_df[
    edge_df["from_address"].isin(use_nodes) &
    edge_df["to_address"].isin(use_nodes)
].copy()

print("Balanced #nodes:", node_df_bal.shape[0],
      "| 1‑class =", node_label_bal.sum(),
      "| 0‑class =", (node_label_bal == 0).sum())

df = df_balanced

In [20]:
# %% [feature engineering]
import pandas as pd

# 0) Предполагаем, что df уже загружен и очищен от NaN/дубликатов

# 1) Извлекаем год из ISO-строки block_timestamp
df['block_year'] = df['block_timestamp'].str[:4].astype(int)

# 2) Категории value (3 квантиля)
df['value_cat'] = pd.qcut(df['value'], q=3, labels=False)

# 3) Считаем node-признаки на уровне адреса
# -------------------------------------------------------------------
# Собираем все вхождения адрес → год первой транзакции
addr_years = pd.concat([
    df[['from_address', 'block_year']].rename(columns={'from_address': 'address'}),
    df[['to_address',   'block_year']].rename(columns={'to_address':   'address'}),
], ignore_index=True)
reg_year = addr_years.groupby('address')['block_year'].min().rename('reg_year')

# Собираем все вхождения адрес → максимальное значение транзакции
addr_vals = pd.concat([
    df[['from_address', 'value']].rename(columns={'from_address': 'address'}),
    df[['to_address',   'value']].rename(columns={'to_address':   'address'}),
], ignore_index=True)
max_value = addr_vals.groupby('address')['value'].max().rename('max_value')

# Склеиваем node-таблицу
node_features_df = (
    pd.concat([reg_year, max_value], axis=1)
      .reset_index()               # index → address
)
node_features_df.columns = ['address', 'reg_year', 'max_value']

# 4) Считаем edge-признаки, мерджим туда годы работы из node-таблицы
# -------------------------------------------------------------------
# Сначала добавим «from_» признаки
edge_df = df[[
    'from_address', 'to_address',
    'gas', 'gas_price',
    'receipt_cumulative_gas_used', 'receipt_gas_used',
    'block_number', 'value_cat',
    'from_scam', 'to_scam'
]].copy()

# прицепляем год регистрации и max_value, затем считаем years_in_operation
edge_df = edge_df.merge(
    node_features_df.rename(columns={
        'address': 'from_address',
        'reg_year': 'from_reg_year',
        'max_value': 'from_max_value'
    }),
    on='from_address', how='left'
)
edge_df = edge_df.merge(
    node_features_df.rename(columns={
        'address': 'to_address',
        'reg_year': 'to_reg_year',
        'max_value': 'to_max_value'
    }),
    on='to_address', how='left'
)

current_year = edge_df['block_year'].max() if 'block_year' in edge_df else df['block_year'].max()
# Но нам нужен block_year в edge_df для расчёта лет в эксплуатации:
edge_df['block_year'] = df['block_year']  # просто копируем
edge_df['from_years_in_operation'] = current_year - edge_df['from_reg_year']
edge_df['to_years_in_operation']   = current_year - edge_df['to_reg_year']

# 5) Итоговые списки признаков
edge_feature_cols = [
    'gas', 'gas_price',
    'receipt_cumulative_gas_used', 'receipt_gas_used',
    'block_number', 'value_cat',
    'from_years_in_operation', 'to_years_in_operation'
]

node_feature_cols = [
    'address',    # ID узла
    'reg_year',   # год регистрации
    'max_value',  # макс. сумма транзакций
]

response_cols = ['from_scam', 'to_scam']

# 6) Проверим, что всё на месте
print("Edge DataFrame shape:", edge_df.shape)
print("Edge feature columns:", edge_feature_cols)
print("Node DataFrame shape:", node_features_df.shape)
print("Node feature columns:", node_feature_cols)
print("Response columns:", response_cols)

# При желании можно отделить чисто две таблицы:
node_df = node_features_df.set_index('address')
edge_df = edge_df[['from_address','to_address'] + edge_feature_cols + response_cols]

Edge DataFrame shape: (10138, 17)
Edge feature columns: ['gas', 'gas_price', 'receipt_cumulative_gas_used', 'receipt_gas_used', 'block_number', 'value_cat', 'from_years_in_operation', 'to_years_in_operation']
Node DataFrame shape: (8819, 3)
Node feature columns: ['address', 'reg_year', 'max_value']
Response columns: ['from_scam', 'to_scam']


# Определяем модель

In [21]:
print("DataFrame columns:", df.columns.tolist())

DataFrame columns: ['hash', 'nonce', 'transaction_index', 'from_address', 'to_address', 'value', 'gas', 'gas_price', 'input', 'receipt_cumulative_gas_used', 'receipt_gas_used', 'block_timestamp', 'block_number', 'block_hash', 'from_scam', 'to_scam', 'from_category', 'to_category', 'any_scam', 'block_year', 'value_cat']


In [ ]:
# ───────────── Подготовка меток и разбиение ─────────────
from sklearn.model_selection import train_test_split
import torch
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# 1) метка «хотя бы одна отметка scam у узла»
from_label = df.groupby("from_address")["from_scam"].max()
to_label = df.groupby("to_address")["to_scam"].max()

node_label = (
    from_label.reindex(node_df.index, fill_value=0).astype(int)
    | to_label.reindex(node_df.index, fill_value=0).astype(int)
)

# 2) train / val / test = 50 / 25 / 25 %, стратифицированно
train_ids, rest_ids = train_test_split(
    node_df.index,
    train_size=0.5,
    stratify=node_label,
    random_state=SEED,
)
val_ids, test_ids = train_test_split(
    rest_ids,
    train_size=0.5,
    stratify=node_label.reindex(rest_ids),
    random_state=SEED,
)

print(
    f"train / val / test sizes: {len(train_ids)} | {len(val_ids)} | {len(test_ids)}")

train / val / test sizes: 4409 | 2205 | 2205


In [ ]:
import torch
import random
import numpy as np
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
from torch_geometric.utils import add_self_loops
from sklearn.metrics import classification_report, confusion_matrix



# 1) ─────────────── Подготовка графа ──────────────────────────
#   node_df     – как в вашем пайплайне (index = address)
#   edge_df     – столбцы ["from_address", "to_address", …]
#   node_label  – Series(address → 0/1, «scam‑узел»)

addr2idx = {addr: i for i, addr in enumerate(node_df.index)}
num_nodes = len(addr2idx)

# Node‑признаки: возьмём два, что вы уже сделали (reg_year, max_value)
x = torch.tensor(
    node_df[["reg_year", "max_value"]].values,
    dtype=torch.float32,
)

# Список рёбер (направленный граф, как в исходном варианте)
edge_index = torch.tensor(
    [
        [addr2idx[s], addr2idx[t]]
        for s, t in zip(edge_df["from_address"], edge_df["to_address"])
        if s in addr2idx and t in addr2idx
    ],
    dtype=torch.long,
).t().contiguous()

# Добавим self‑loops (помогает сходиться GraphSAGE)
edge_index, _ = add_self_loops(edge_index, num_nodes=num_nodes)

# Массив меток узлов
y = torch.tensor(node_label.values, dtype=torch.long)

# ─ Split на train / val / test (адреса у вас уже выбраны) ────


def mask_from_ids(id_list):
    m = torch.zeros(num_nodes, dtype=torch.bool)
    m[[addr2idx[a] for a in id_list]] = True
    return m


train_mask = mask_from_ids(train_ids)
val_mask = mask_from_ids(val_ids)
test_mask = mask_from_ids(test_ids)

data = Data(
    x=x, edge_index=edge_index, y=y,
    train_mask=train_mask, val_mask=val_mask, test_mask=test_mask,
)

# 2) ─────────────── Определение модели ───────────────────────


class GraphSAGE(torch.nn.Module):
    def __init__(self, in_ch, dropout=0.5):
        super().__init__()
        self.convs = torch.nn.ModuleList([
            SAGEConv(in_ch, 4),
            SAGEConv(4, 32, aggr="mean"),
            SAGEConv(32, 16, aggr="mean"),
            SAGEConv(16, 16, aggr="mean"),
            SAGEConv(16, 16, aggr="mean"),
            SAGEConv(16,  4, aggr="mean"),
        ])
        self.lin = torch.nn.Linear(4, 2)          # 2 класса
        self.dropout = dropout

    def forward(self, x, edge_index):
        for conv in self.convs:
            x = conv(x, edge_index)
            x = torch.relu(x)
            x = torch.nn.functional.dropout(
                x, p=self.dropout, training=self.training
            )
        return torch.log_softmax(self.lin(x), dim=-1)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GraphSAGE(in_ch=data.num_features).to(device)
data = data.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)
loss_fn = torch.nn.NLLLoss()

print(model)

GraphSAGE(
  (convs): ModuleList(
    (0): SAGEConv(2, 4, aggr=mean)
    (1): SAGEConv(4, 32, aggr=mean)
    (2): SAGEConv(32, 16, aggr=mean)
    (3-4): 2 x SAGEConv(16, 16, aggr=mean)
    (5): SAGEConv(16, 4, aggr=mean)
  )
  (lin): Linear(in_features=4, out_features=2, bias=True)
)


# Обучение модели

In [25]:
# 3) ─────────────── Обучение ─────────────────────────────────
EPOCHS = 50
for epoch in range(1, EPOCHS + 1):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    # валидация
    model.eval()
    with torch.no_grad():
        pred = out.argmax(dim=1)
        train_acc = (pred[data.train_mask] ==
                     data.y[data.train_mask]).float().mean().item()
        val_acc = (pred[data.val_mask] == data.y[data.val_mask]
                   ).float().mean().item()

    if epoch == 1 or epoch % 5 == 0:
        print(f"Epoch {epoch:02d} | loss {loss:.4f} | "
              f"train_acc {train_acc:.4f} | val_acc {val_acc:.4f}")

# 4) ─────────────── Тест + отчёты ────────────────────────────
model.eval()
with torch.no_grad():
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    test_acc = (pred[data.test_mask] ==
                data.y[data.test_mask]).float().mean().item()

print(f"\nTEST accuracy: {test_acc:.4f}")
print("\nClassification report:")
print(
    classification_report(
        data.y[data.test_mask].cpu(), pred[data.test_mask].cpu(), digits=4
    )
)
print("Confusion matrix:\n",
      confusion_matrix(
          data.y[data.test_mask].cpu(), pred[data.test_mask].cpu()
      ))

Epoch 01 | loss 258697906146508800.0000 | train_acc 0.5566 | val_acc 0.5510
Epoch 05 | loss 141135442845106176.0000 | train_acc 0.5845 | val_acc 0.6127
Epoch 10 | loss 86533712589619200.0000 | train_acc 0.6723 | val_acc 0.6363
Epoch 15 | loss 103958867355893760.0000 | train_acc 0.6825 | val_acc 0.6739
Epoch 20 | loss 67602076569436160.0000 | train_acc 0.6990 | val_acc 0.7256
Epoch 25 | loss 31649714935955456.0000 | train_acc 0.7308 | val_acc 0.7234
Epoch 30 | loss 51510899990593536.0000 | train_acc 0.7410 | val_acc 0.7401
Epoch 35 | loss 61190369001340928.0000 | train_acc 0.7512 | val_acc 0.7370
Epoch 40 | loss 29844447839649792.0000 | train_acc 0.7845 | val_acc 0.7837
Epoch 45 | loss 48303117766230016.0000 | train_acc 0.7848 | val_acc 0.7678
Epoch 50 | loss 43148005959794688.0000 | train_acc 0.7802 | val_acc 0.7932

TEST accuracy: 0.9814

Classification report:
              precision    recall  f1-score   support

           0     0.9814    1.0000    0.9906      2164
           1    

d:\Fraud\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Fraud\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Fraud\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
